## **Notebook for aggregating sequencing reports**

### **Packages**

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import glob, os, re
import matplotlib.pyplot as plt
from datetime import  datetime
from ipywidgets import widgets, interactive
from pandas import ExcelWriter

In [2]:
dt = datetime.today().strftime(format='%d-%m-%Y')

### **Directories & Files**
Uniqueness in directory and file names is assumed for all analyses

The organisation of the `run_dir`: The directory name MUST be unique and reside anywhere inside `sars_dir` directory
 
| Directory name | File name | File source-tool | File description |
| :-------------- | :--------- | :---------------- | :------|
|qcs *(QCs)*|`*.tsv`|Quast|Transposed report|
|nxt *(nextclade)*|`nxt.tsv`|Nextclade|Renamed `nextclade.tsv` output|
|snpEff|`*sneff.vcf`| snpEff|Per-sample unzipped `snpEff` output|
|var *(variants)*|`k-per-gene_variant_anns.tsv`|Script: `abstract_snpeff_ann_output.py`|Aggregation of individual `snpEff.vcf` outputs by abstracting gene-mutations|
|png *(pangolin)*|`png.csv`|Pangolin|Renamed `Pangolin result.csv` output|
|dpt *(coverage depths)*|`amplicon, genome`|Mosdepth|Per-sample amplicon/genome depths<br> ***Cols**:chrom, start, end, region, coverage, sample*|
|plt *(plot)*|`*.pdf`|Quast|Heatmap plot for median read depth coverage|

### **Preliminary variables**

In [3]:
sars_dir = 'SARS-CoV-2' #  name of root directory for all SARS-associated work
home_dir = os.getenv('HOME') #  get OS home directory
parent_dir = glob.glob(f'{home_dir}/**/{sars_dir}', recursive=True)[0]

### **Metadata**

In [4]:
# import raw metadata file
df_rmd_cln = pd.read_excel(glob.glob(f'{parent_dir}/**/Outputs/COVID19-resultsCts-merged-cln.xlsx', recursive=True)[0]).rename(columns={'S_NUM': 'sample_name'})

### **Functions**

In [5]:
# define a func to replace spaces in the header names
def tidy_header(df):
    df.columns = [col.replace(' ', '_') for col in df.columns]
    return df

In [6]:
# define func to retrieve particular columns from a df (spaces in col names must be replaced with _ in the input col_list)
def get_cols(df, col_list):
    new_df = tidy_header(df)
    return new_df[col_list]

In [7]:
# define a function to replace from a dictionary ('key is what is to be replaced': 'value is the replacement')
def replace(string, substitutions):
    substrings = sorted(substitutions, key=len, reverse=True)
    regex = re.compile('|'.join(map(re.escape, substrings)))
    return regex.sub(lambda match: substitutions[match.group(0)], string)

In [8]:
# define function to merge variants and nextclade data
def merge_varNxt(df_var_cln, df_nxt_cln):
    return (df_var_cln.set_index('sample_name').merge(df_nxt_cln
          .set_index('seqName'), how='outer', left_index=True, right_index=True)
                 .reset_index().rename(columns={'index': 'sample_name'}))

In [9]:
# define function to merge pangolin and variants-nextclade data
def merge_pngVxt(df_png_cln, df_varNxt):
    return (df_png_cln.set_index('Sequence_name').merge(df_varNxt
        .set_index('sample_name'), how='outer', left_index=True, right_index=True)
            .reset_index().rename(columns={'index': 'Sequence_name'}))


In [10]:
# define function to merge metadata with cts data
def merge_rmdCts(df_rmd_cln, df_cts_cln):
    return (df_rmd_cln.set_index('S_NUM').merge(df_cts_cln.set_index('Sample_Name'), how='outer', left_index=True, right_index=True)
            .reset_index().rename(columns={'index': 'S_NUM'}))


In [11]:
# define function to merge metadata and seq data
def merge_vnpPmd(df_pngVxt, df_rmdCts):
    return (df_pngVxt.set_index('S_NUM')
            .merge(df_rmdCts.set_index('S_NUM'), how='left', left_index=True, right_index=True)
                 .reset_index().rename(columns={'index': 'S_NUM'}))

In [12]:
# define a function to retrieve MoC and all mutations for the s-gene
def get_mut_of_concern(ann_file_name, moc_list):

    def intersection(x, y):
        return list(set(x) & set(y))

    moc_list = moc_list
#     file_name = 'k-per-gene_variant_anns.tsv'
    df = ann_file_name[['sample_name','S']]

#     df = pd.read_table(f'{base_dir}/{file_name}')[['sample_name','S']]
    mutations = []
    moc = []
    sample_id = []
    for row in df.itertuples():
        if isinstance(row.S, str):
            sgene = row.S
        else: 
            sgene = str(row.S)
        substitutions = sgene.replace(' ', '').split(',')[1:-1]
        if len(moc_list) >= len(intersection(moc_list, substitutions)) > 0:
            intsct = intersection(moc_list, substitutions)
            sample_name = row.sample_name
            mutations.append(str(substitutions).replace("[", "").replace("]", "").replace("'", ""))
            moc.append(str(intsct).replace("[", "").replace("]", "").replace("'", ""))
            sample_id.append(sample_name)
        else: pass 
    df = pd.DataFrame({'Sample_ID': sample_id, 'Mut_of_Concern_(S)': moc, 'All_Mutations_(S)': mutations})
    df_fnl = df.assign(Sample_ID = df['Sample_ID'].apply(lambda x: x.split('_')[0].split('.')[0]))
    return df_fnl

In [13]:
def replace_with_who_lin(x):
    if x == 'B.1.1.7':
        return x.replace(x, 'B.1.1.7(Alpha)')
    elif x == 'B.1.617.2':
        return x.replace(x, 'B.1.617.2(Delta)')
    elif x == 'B.1.351':
        return x.replace(x, 'B.1.351(Beta)')
    elif x == 'B.1.525':
        return x.replace(x, 'B.1.525(Eta)')
    elif 'AY' in str(x):
        return str(x).replace(str(x), str(x)+'(Delta)')
    elif x == 'B.1.1.529':
        return x.replace(x, 'B.1.1.529(Omicron)')
    elif 'BA' in str(x):
        return str(x).replace(str(x), str(x)+'(Omicron)')
    return x   

### **Variables**

Reassign accordingly...

In [47]:
pipeline = 'nf-viralrecon-v2.2' #  name and version
seq_name = 'seq34' #  seq*
tech = 'NextSeq' #  NextSeq/MiSeq/MinION
seq_dt = '24/01/2022' #  DD/MM/YYYY
lib_prep = 'COVIDSeq' #  NEBNext/NEBNext_FS/COVIDSeq/Nextera_XT
primer_set = 'ARTIC_V3' #  ARTIC_V3/ARTIC_V4
identifier = 'ILL_seq34'#  used in naming file outputs
run_dir = 'output_2022-01-24_run34_nextseq' #  name of the run directory containing viralcon pipeline output as implemented by Kibet

### **Sequencing sheet**

In [48]:
# import sequencing cheat sheet
df_seq_sh = (pd.read_excel(glob.glob(f'{parent_dir}/**/SeqSampleSheets/index_cheat_sheet_{seq_name}.xlsx', recursive=True)[0], usecols=['indexing', 'plt_pos']).
             rename(columns={'indexing': 'sample_name'}))
df_seq_sh_fnl = df_seq_sh[df_seq_sh['sample_name'].str.contains('COV')]

### **QCstats**

In [49]:
# import the collated file for all the multiqc output
df_qcs_trans_cols = ['sample_name', 'Genome fraction (%)']# , 'Assembly'
df_qcs_trans = pd.read_table(glob.glob(f'{parent_dir}/**/{run_dir}/qcs/transposed_report.tsv', recursive=True)[0])
df_qcs_trans2 = df_qcs_trans.assign(sample_name = df_qcs_trans['Assembly'].apply(lambda x: x.split('_')[0].split('.')[0]))[df_qcs_trans_cols].rename(columns={'Genome fraction (%)': 'genome_cov'})
df_qcs_trans_fnl = df_qcs_trans2.assign(genome_cov=df_qcs_trans2['genome_cov'].replace('-', np.nan).apply(lambda x: round(float(x),1) if float(x) else np.nan)).rename(columns={'genome_cov': 'Genome fraction (%)'})
# df_qcs_trans_fnl['Seq id'] = run_dir
# df_qcs_fnl['Analysis type'] = 'Qcs'

In [50]:
df_qcs_trans_fnl.head(1)

,sample_name,Genome fraction (%)
0,COVC26122,32.2


In [51]:
# import the collated file for all the multiqc output
if 'Seq' in tech:
    df_qcs_cols = ['sample_name', '# Input reads', '# Trimmed reads (fastp)',
       '% Mapped reads', '# Mapped reads', '# Trimmed reads (iVar)', 
       'Coverage median', '% Coverage > 1x', '% Coverage > 10x',
       'Pangolin lineage (iVar)', 'Nextclade clade (iVar)']#  , 'Sample'
else:
    df_qcs_cols = ['Sample', '# Mapped reads', 'Coverage median', '% Coverage > 1x', '% Coverage > 10x',
       'Pangolin lineage', 'Nextclade clade']

df_qcs = pd.read_csv(glob.glob(f'{parent_dir}/**/{run_dir}/qcs/summary_variants_metrics_mqc.csv', recursive=True)[0], sep=',')
df_qcs_fnl = df_qcs.assign(sample_name = df_qcs['Sample'].apply(lambda x: '_'.join(x.split('_')[:-1]) if (len(x.split('_')) > 2) else x.split('_')[0]))[df_qcs_cols]
if tech == 'MinION':
    df_qcs_fnl[['# Input reads', '# Trimmed reads (fastp)', '% Mapped reads', '# Trimmed reads (iVar)']] = [np.nan, np.nan, np.nan, np.nan]
    df_qcs_fnl = df_qcs_fnl[['Sample', '# Input reads', '# Trimmed reads (fastp)',
       '% Mapped reads', '# Mapped reads', '# Trimmed reads (iVar)', 
       'Coverage median', '% Coverage > 1x', '% Coverage > 10x',
       'Pangolin lineage', 'Nextclade clade']].rename(columns={'Sample': 'sample_name'})
df_qcs_fnl['Seq id'] = run_dir
# df_qcs_fnl['Analysis type'] = 'Qcs'

In [52]:
df_qcs_fnl.head(1)

,sample_name,# Input reads,# Trimmed reads (fastp),% Mapped reads,# Mapped reads,# Trimmed reads (iVar),Coverage median,% Coverage > 1x,% Coverage > 10x,Pangolin lineage (iVar),Nextclade clade (iVar),Seq id
0,COVC26081,890566,626358,92.26,577890,577304.0,2308.0,100.0,100.0,B.1.1,20B,output_2022-01-24_run34_nextseq


In [53]:
df_seq_qcs = df_seq_sh_fnl.merge(df_qcs_trans_fnl, left_on='sample_name', right_on='sample_name', how='left').sort_values('Genome fraction (%)', ascending=False)

In [54]:
df_seq_qcs.head(1)

,sample_name,plt_pos,Genome fraction (%)
102,COVM02498,G1,99.8


In [55]:
qcStat = df_seq_qcs.merge(df_qcs_fnl, left_on='sample_name', right_on='sample_name', how='left').sort_values('Genome fraction (%)', ascending=False)

In [56]:
df_seq_meta = df_seq_qcs.merge(df_rmd_cln, left_on='sample_name', right_on='sample_name', how='left').sort_values('Genome fraction (%)', ascending=False)
# df_seq_meta.head()

In [57]:
qcStat.head(1)

,sample_name,plt_pos,Genome fraction (%),# Input reads,# Trimmed reads (fastp),% Mapped reads,# Mapped reads,# Trimmed reads (iVar),Coverage median,% Coverage > 1x,% Coverage > 10x,Pangolin lineage (iVar),Nextclade clade (iVar),Seq id
0,COVM02498,G1,99.8,963860.0,683044.0,92.67,632966.0,632384.0,2470.0,100.0,100.0,B.1.1,20B,output_2022-01-24_run34_nextseq


### **Nextclade data**

In [58]:
# import Nextclade clade data
df_nxt = pd.read_table(glob.glob(f'{parent_dir}/**/{run_dir}/nxt/nxt.tsv', recursive=True)[0])

# sep = '.' if 'Seq' in tech else '_'

# coverage based on totalMissing col in nexclade output
coverage = round(100 - (df_nxt['totalMissing'] / 29903) * 100, 1)
df_nxt_cln1 = df_nxt.assign(seqName = df_nxt['seqName'].apply(lambda x: x.split('_')[1].split('.')[0] if len(x.split('_')) > 2 else x.split('/')[0])).rename(columns={'seqName': 'sample_name'})
df_nxt_cln2 = df_nxt_cln1.assign(coverage = coverage)
df_nxt_fnl = df_nxt_cln1
df_nxt_fnl['Seq id'] = run_dir
# df_nxt_fnl['Analysis type'] = 'Nxt'

In [59]:
nextclade = df_seq_qcs.merge(df_nxt_fnl, left_on='sample_name', right_on='sample_name', how='left').sort_values('Genome fraction (%)', ascending=False)
nextclade.head(1)

,sample_name,plt_pos,Genome fraction (%),clade,qc.overallScore,qc.overallStatus,totalSubstitutions,totalDeletions,totalInsertions,totalFrameShifts,...,qc.frameShifts.frameShiftsIgnored,qc.frameShifts.totalFrameShiftsIgnored,qc.frameShifts.score,qc.frameShifts.status,qc.stopCodons.stopCodons,qc.stopCodons.totalStopCodons,qc.stopCodons.score,qc.stopCodons.status,errors,Seq id
0,COVM02498,G1,99.8,21K (Omicron),10.5625,good,54.0,39.0,9.0,0.0,...,NaN,0.0,0.0,good,NaN,0.0,0.0,good,NaN,output_2022-01-24_run34_nextseq


### **Variants data**

In [60]:
# import the collated file for all the snpEff outputs
df_var = pd.read_csv(glob.glob(f'{parent_dir}/**/{run_dir}/var/k-per-gene_variant_anns.tsv', recursive=True)[0], sep='\t')
df_var_fnl = df_var.assign(sample_name = df_var['sample_name'].apply(lambda x: x.split('_')[0].split('.')[0]))
# df_var_fnl['Seq id'] = df_var_fnl.sample_name.
# df_var_fnl['Analysis type'] = 'Var'

In [61]:
df_var_fnl.head(1)

,sample_name,num_vars,ORF1ab,ORF1a,S,ORF3a,ORF3b,E,M,ORF6,ORF7a,ORF7b,ORF8,N,ORF9a,ORF9b,ORF10
0,COVM02672,56,"NC, Q607H, K856R, F924F, V1393V, A1707A, S2083...",NaN,"A67V, H69_V70del, T95I, G142_Y145delinsD, G339...",T64T,NaN,T9I,"D3G, Q19E, A63T",R20R,NaN,NaN,"NC, E106*","NC, P13L, E31_S33del, R203K, R203R, G204R",NaN,NaN,NaN


In [62]:
iVar_snpE = df_seq_qcs.merge(df_var_fnl, left_on='sample_name', right_on='sample_name', how='left').sort_values('Genome fraction (%)', ascending=False)
iVar_snpE.head(1)

,sample_name,plt_pos,Genome fraction (%),num_vars,ORF1ab,ORF1a,S,ORF3a,ORF3b,E,M,ORF6,ORF7a,ORF7b,ORF8,N,ORF9a,ORF9b,ORF10
0,COVM02498,G1,99.8,64.0,"NC, NC, K856R, F924F, A1707A, S2083_L2084delin...",NaN,"A67V, H69_V70del, T95I, G142_Y145delinsD, N211...",T64T,NaN,T9I,"D3G, Q19E, A63T",R20R,NaN,NaN,NC,"NC, P13L, E31_S33del, R203K, R203R, G204R",NaN,NaN,NaN


### **Pangolin data**

In [63]:
# import Pangolin lineage data
# df_png = pd.read_csv(glob.glob(f'{parent_dir}/**/{run_dir}/png/png.csv', recursive=True)[0])
df_png_web = pd.read_csv(glob.glob(f'{parent_dir}/**/{run_dir}/png/png.csv', recursive=True)[0]).rename(columns={'Sequence name': 'sample_name'})

months = {'January': 'Jan', 'February': 'Feb', 'March': 'Mar',
         'April': 'Apr', 'June': 'Jun', 'July': 'Jul', 'August': 'Aug',
          'September': 'Sep', 'October': 'Oct', 'November': 'Nov', 'December': 'Dec'}
# retrieve cols Sequence_name and Lineage (func get_cols replaces col names spaces with _)
# cols = ['taxon', 'lineage', 'scorpio_call']#, 'Most_common_countries']
# df_png_cln = get_cols(tidy_header(df_png), cols)
# df_png_fnl = (df_png_cln.assign(taxon = df_png_cln['taxon']
#             .apply(lambda x: '_'.join(x.split('_')[1:2]) if (len(x
#             .split('_')) > 2) else x.split('_')[0]))).rename(columns={'taxon': 'sample_name'})
df_png_fnl_web = (df_png_web.assign(sample_name = df_png_web['sample_name']
                .apply(lambda x: x.split('_')[1].split('.')[0] if len(x.split('_')) > 2 else x.split('/')[0])))
# df_png_fnl = df_png_cln1.assign(Date_range=df_png_cln1['Date_range'].apply(lambda x: replace(x, months) if (isinstance(x, str)) else x))
df_png_fnl_web['Seq id'] = run_dir
# df_png_fnl_web['Analysis type'] = 'Png'

In [64]:
pangolin = df_seq_qcs.merge(df_png_fnl_web, left_on='sample_name', right_on='sample_name', how='left').sort_values('Genome fraction (%)', ascending=False)
pangolin.head(1)

,sample_name,plt_pos,Genome fraction (%),Lineage,Conflict,Ambiguity score,Scorpio call,Scorpio support,Scorpio conflict,Note,pangolin version,pangoLEARN version,Seq id
0,COVM02498,G1,99.8,BA.1.1,0.0,0.999371,Omicron (BA.1-like),0.8966,0.0,scorpio call: Alt alleles 52; Ref alleles 0; A...,3.1.17,2022-01-20,output_2022-01-24_run34_nextseq


### **Summary: combining data**

#### *Merge summary data*

In [65]:
pango_summ = pangolin[['sample_name', 'Lineage', 'Scorpio call']]
next_summ = nextclade[['sample_name', 'clade']]
var_s = iVar_snpE[['sample_name', 'S']]
var_summ = var_s.assign(spike_mut = var_s['S'].map(lambda x: int(len(x.split(',')))  if isinstance(x, str) else 0))
pango_next = pango_summ.merge(next_summ, left_on='sample_name', right_on='sample_name', how='outer')
pn_var = pango_next.merge(var_summ, left_on='sample_name', right_on='sample_name', how='outer')
meta_summ = df_seq_meta[['CASE_ID', 'sample_name']]

df_comb = df_seq_qcs.merge(pn_var, left_on='sample_name', right_on='sample_name', how='left')
summary = (meta_summ.merge(df_comb, left_on='sample_name', right_on='sample_name', how='right').sort_values('Genome fraction (%)', ascending=False)
          .rename(columns={'CASE_ID': 'Case id', 'sample_name': 'Unique lab id', 'plt_pos': 'Seq plate pos', 'clade': 'Clade', 'S': 'Spike mutations (snpEff)', 'spike_mut':'Spike mutation count'}))

summary[['Seq number', 'Seq platform', 'Seq date', 'Library kit', 'Primer set', 'Analysis pipeline']] = [seq_name, tech, seq_dt, lib_prep, primer_set, pipeline]

In [66]:
summary.head(1)

,Case id,Unique lab id,Seq plate pos,Genome fraction (%),Lineage,Scorpio call,Clade,Spike mutations (snpEff),Spike mutation count,Seq number,Seq platform,Seq date,Library kit,Primer set,Analysis pipeline
0,MCRH/MIG19712/2021,COVM02498,G1,99.8,BA.1.1,Omicron (BA.1-like),21K (Omicron),"A67V, H69_V70del, T95I, G142_Y145delinsD, N211...",35,seq34,NextSeq,24/01/2022,COVIDSeq,ARTIC_V3,nf-viralrecon-v2.2


### **Generate reports**

In [67]:
writer = pd.ExcelWriter(f"{glob.glob(f'{parent_dir}/**/SeqReports', recursive=True)[0]}/{run_dir}.Analysis.QCstats.xlsx")
datasheets = [(qcStat, 'QCstats'), (pangolin, 'pangolinAnalysis'), (nextclade, 'nextcladeAnalysis'), (iVar_snpE, 'snpEffAnnotation'), (df_seq_meta, 'metaData'), (summary, 'summaryReport')]
for datasheet in datasheets:
    samp_id = 'sample_name' if datasheet[1] != 'summaryReport' else 'Unique lab id'
    (datasheet[0].sort_values([samp_id, 'Genome fraction (%)'], ascending=[True, False]).drop_duplicates(samp_id, keep='first').sort_values('Genome fraction (%)', ascending=False)
     .to_excel(writer, sheet_name=datasheet[1], index=False, na_rep='NA', float_format='%.1f'))

writer.save()

### **County feedback data**

In [68]:
df_counties = df_seq_meta[['CASE_ID', 'sample_name', 'COUNT_RES']]

# unique county identifiers
reports = [('Homabay', 'HCRH'), ('Migori', 'MCRH'), ('Kisii', 'KCRH'), 
           ('Nyamira', 'NCRH'), ('Siaya', 'SCRH'), ('KCSS'), ('Bukavu', 'DRC03'),
           ('Siaya', 'SIAYA'), ('Bungoma', 'BCRH'), ('Kapenguria', 'WPKRH'),
           ('Kitale', 'TNZ'), ('Busia', 'BSCRH')]
codes = set(df_counties.CASE_ID.map(lambda x: str(x).split('/')[0]))

for report in reports:
    mask1 = df_counties['COUNT_RES'] == report[0]
    mask2 = df_counties['CASE_ID'].str.contains(report[1]) == True
    mask3 = df_counties['CASE_ID'].str.contains(report[0]) == True
#     print(report)
    if len(report) != 2 and report[0] in codes:
        df_report = df_counties[mask3 == True]['sample_name']
        writer = pd.ExcelWriter(f"{glob.glob(f'{parent_dir}/**/CountyFeedbacks', recursive=True)[0]}/{run_dir}_{report}.Analysis.QCstats.xlsx")
    elif len(report) == 2 and report[1] not in codes: 
        pass
    
    elif len(report) == 2 and report[1] in codes:
        df_report = df_counties[mask2 == True]['sample_name']
        writer = pd.ExcelWriter(f"{glob.glob(f'{parent_dir}/**/CountyFeedbacks', recursive=True)[0]}/{run_dir}_{report[0]}.Analysis.QCstats.xlsx")
        for datasheet in datasheets:
            samp_id = 'sample_name' if datasheet[1] != 'summaryReport' else 'Unique lab id'
            (datasheet[0][datasheet[0][samp_id].isin(df_report)].sort_values([samp_id, 'Genome fraction (%)'], ascending=[True, False]).drop_duplicates(samp_id, keep='first')
             .sort_values('Genome fraction (%)', ascending=False)
             .to_excel(writer, sheet_name=datasheet[1], index=False, na_rep='NA', float_format='%.1f'))
        writer.save()    